In [ ]:
"""Cell 1: Parse all text files from AH using dc.BColl and save as a 'flat' json
Output will be log messages of events encountered during parsing of files"""
import dc
bc = dc.BColl(parse = True)
bc.deleteRawTS()
bc.deleteIfNot()
bc.writeFlatDict(fileName = 'all_en_ts_flat_dump.json')

In [ ]:
"""Cell 2: Read the flat json into a dc2.PColl class, then parse and add xls(x) 
files from e-rapport and landlord to these
Note that the e-rapport files will cause many warnings. This is because there are 
images embedded in the files that confuses the xlrd library, it is not a problem"""
import json, re, os, traceback, datetime, copy, csv, xlrd, 
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import dc, dc2, dc2u
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas import DataFrame as pddf
oneHour = datetime.timedelta(seconds=3600) # Global shorthand, used in later cells
b = dc2.PColl(source = 'all_en_ts_flat_dump.json')
b.parseMetryFiles()
b.parseLandlordFiles()
b.parseERapportFiles()
with open('PColl_b.pkl','wb') as oFile: pickle.dump(b, oFile, -1)

In [ ]:
"""Cell 3: Quickly load 'b' via pickle if no new parses need to be done"""
import pickle
with open('PColl_b.pkl','rb') as iFile: b = pickle.load(iFile)

In [ ]:
"""Cell 4: Define all areas and included meters for the three 'areas' (UFOn)
Then, consolidate and merge timeseriess from 'b' (defined in cell 2) into these areas"""
aFj = list([ dict(name = 'Fysik origo', idr = 'O0007001', meters = [['O0007001', 'fj_901',1],
                                                                    ['O0007003', 'fj_901', 1]]),
    #dict(name = 'O7:2', idr = 'O0007002', meters = [['O0007001', 'fj_901',0.08]]),
    #dict(name = 'O7:3', idr = 'O0007003', meters = [['O0007001', 'fj_901',0.41]]),
    dict(name = 'Polymerteknologi', idr = 'O0007005', meters = [['O0007005', 'fj_902', 1],
                                                                ['O0007005', 'fj_901', 1]]),
    dict(name = 'Nya Matte', idr = 'O0007006', meters = [['O0007006', 'fj_901', 1]]),
    dict(name = 'Elkraftteknik', idr = 'O0007012', meters = [['O0007012', 'fj_901', 1],
                                                            ['O0007012', 'fj_902', -1]]),
    dict(name = 'Gibraltar herrgrd', idr = 'O0007014', meters = [[]]),
    dict(name = 'Chalmers bibliotek', idr = 'O0007017', meters = [['O0007017', 'fj_901', 1]]),
    dict(name = 'Idelra', idr = 'O0007018', meters = [['O0007018', 'fj_901', 1]]),
    dict(name = 'Centrala administrationen', idr = 'O0007019', meters = [['O0007019', 'fj_901', 1]]),
    dict(name = 'Hrsalar HC', idr = 'O0007021', meters = [['O0007021', 'fj_901', 1]]),
    dict(name = 'Hrsalar HA', idr = 'O0007022', meters = [['O0007022', 'fj_901', 1]]),
    dict(name = 'Vg och vatten 1', idr = 'O0007023', meters = [['O0007023', 'fj_901', 1]]),
    dict(name = 'EDIT', idr = 'O0007024', meters = [['O0007024', 'fj_901', 1],
                                                    ['O0007024', 'fj_902', 1],
                                                   ['O0007024', 'fj_903', 1],
                                                    ['O0007024', 'fj_904', 1],
                                                   ['O0007024', 'fj_905', 1]]),
    dict(name = 'Hrsalar HB', idr = 'O0007025', meters = [['O0007025', 'fj_901', 1]]), # Ignore fj_902
    dict(name = 'Arkitektur', idr = 'O0007026', meters = [['O0007026', 'fj_901', 1]]),
    dict(name = 'Vg och vatten 2', idr = 'O0007027', meters = [['O0007027', 'fj_901', 1]]),
    dict(name = 'Maskinteknik', idr = 'O0007028', meters = [['O0007028', 'fj_901', 1], 
                                                            ['O0007888', 'fj_001', -1],
                                                           ['O0007017', 'fj_901', -1], 
                                                            ['O0007028', 'fj_902', -1]]),
    dict(name = 'Gamla Matte', idr = 'O0007040', meters = [['O0007040', 'fj_901', 1]]),
    dict(name = 'P-hus Campus Johanneberg', idr = 'O0007043', meters = [[]]),
    dict(name = 'Lokalkontor AH', idr = 'O0007888', meters = [['O0007888', 'fj_001', 1]]),
    dict(name = 'Fysik Soliden', idr = 'O0011001', meters = [['O0011001', 'fj_901', 1]]),
    dict(name = 'Keramforskning', idr = 'O0013001', meters = [['O0013001', 'fj_901', 1], 
                                                              ['O0007005', 'fj_901', -1],
                                                             ['O0007005', 'fj_902', -1]]),
    dict(name = 'Kemiteknik', idr = 'O3060132', meters = [['O3060132', 'fj_001', 0.952]]), # Meter scaling to remove Gym
    dict(name = 'Fysik MC2', idr = 'O3060133', meters = [['O3060133', 'fj_001', 1]]),
    dict(name = 'Krhuset', idr = 'O3060150_1', meters = [[]]),
    dict(name = 'CTP', idr = 'O3060137', meters = [['O3060137', 'fj_30003859', 1]]),
    dict(name = 'JSP', idr = 'O3060138', meters = [['O3060138', 'fj_VS1VMM1', 1]]),
    dict(name = 'Vasa 1', idr = 'O3060101', meters = [['O3060101', 'fj_30003857', 1]]),
    dict(name = 'Vasa 2-3', idr = 'O3060102_3', meters = [['O3060102', 'fj_30003856', 1]]),
    dict(name = 'Vasa 4-15', idr = 'O3060104_15', meters = [['O3060104', 'fj_30003855', 1]]),
    dict(name = 'Chabo', idr = 'O4002700', meters = [['O4002700', 'fj_30003858', 1]])
    #dict(name = 'Fysik Origo 3', idr = 'O0007003', meters = [['O0007003', 'fj_901', 1]])
         ])

"""Electricity UFOn"""
aEl = list([ dict(name = 'Fysik origo', idr = 'O0007001', meters = [['O0007001', 'el_901',1],
                                                                    ['O0007001', 'el_902',1]]),
    dict(name = 'Polymerteknologi', idr = 'O0007005', meters = [['O0007005', 'el_901', 1]]),
    dict(name = 'Nya Matte', idr = 'O0007006', meters = [['O0007006', 'el_901', 1],
                                                         ['O0007006', 'el_902', 1],
                                                         ['O0007017', 'el_901', -1]]),
    dict(name = 'Elkraftteknik', idr = 'O0007012', meters = [['O0007012', 'el_901', 1],
                                                             ['O0007012', 'el_902', -1],
                                                             ['O0007012', 'el_903', 1],
                                                             ['O0007012', 'el_904', 1]]),
    dict(name = 'Gibraltar Herrgrd', idr = 'O0007014', meters = [['O0007014', 'el_901', 1]]),
    dict(name = 'Chalmers bibliotek', idr = 'O0007017', meters = [['O0007017', 'el_921', 1]]),
    dict(name = 'Idelra', idr = 'O0007018', meters = [['O0007018', 'el_901', 1],
                                                      ['O0007018', 'el_902', 1]]),
    dict(name = 'Centrala administrationen', idr = 'O0007019', meters = [['O0007019', 'el_901', 1]]),
    dict(name = 'Hrsalar HC', idr = 'O0007021', meters = [['O0007021', 'el_901', 1]]),
    dict(name = 'Hrsalar HA', idr = 'O0007022', meters = [['O0007022', 'el_901', 1]]),
    dict(name = 'Vg och vatten 1', idr = 'O0007023', meters = [['O0007023', 'el_901', 1],
                                                               ['O0007023', 'el_902', 1],
                                                               ['O0007023', 'el_920', -1],
                                                               ['O0007023', 'el_921', -1]]),
    dict(name = 'EDIT', idr = 'O0007024', meters = [['O0007024', 'el_901', 1], 
                                                    ['O0007024', 'el_902', 1], 
                                                    ['O0007024', 'el_903', 1], 
                                                    ['O0007024', 'el_904', 1], 
                                                    ['O0007024', 'el_905', 1],
                                                    ['O0007024', 'el_906', 1],
                                                    ['O0007024', 'el_907', 1]]),
    dict(name = 'Hrsalar HB', idr = 'O0007025', meters = [['O0007025', 'el_901', 1],
                                                          ['O0007025', 'el_902', 1]]), # Ignore fj_902
    dict(name = 'Arkitektur', idr = 'O0007026', meters = [['O0007026', 'el_901', 1],
                                                          ['O0007026', 'el_903', 1]]),
    dict(name = 'Vg och vatten 2', idr = 'O0007027', meters = [['O0007027', 'el_902', 1],
                                                               ['O0007027', 'el_903', 1],
                                                               ['O0007027', 'el_904', -1],
                                                               #['O0007027', 'el_222', -1],
                                                               ['O0007043', 'el_901', -1],
                                                               ['O0007043', 'el_902', -1]]),
    dict(name = 'Maskinteknik', idr = 'O0007028', meters = [['O0007008', 'el_101', -1], 
                                                            ['O0007008', 'el_102', -1],
                                                            #['O0007028', 'el_006', -1], 
                                                            ['O0007028', 'el_901', 1], 
                                                            ['O0007028', 'el_902', 1], 
                                                            ['O0007028', 'el_903', 1], 
                                                            ['O0007028', 'el_904', 1], 
                                                            ['O0007028', 'el_905', 1]]),
    dict(name = 'Gamla Matte', idr = 'O0007040', meters = [['O0007040', 'el_901', 1]]),
    dict(name = 'Phus Campus Johanneberg', idr = 'O0007043', meters = [['O0007043', 'el_901', 1],
                                                                       ['O0007043', 'el_902', 1]]),
    dict(name = 'Lokalkontor AH', idr = 'O0007888', meters = [['O0007888', 'el_001', 0.95]]),
    dict(name = 'Fysik Soliden', idr = 'O0011001', meters = [['O0011001', 'el_901', 1]]),
    dict(name = 'Keramforskning', idr = 'O0013001', meters = [['O0013001', 'el_901', 1], 
                                                              ['O0007005', 'el_901', -1],
                                                              ['O0013001', 'el_903', -1]]),
    dict(name = 'Kemi- och bioteknik, cfab', idr = 'O3060132', meters = [['O3060132', 'el_009', 1],
                                                                         ['O3060132', 'el_010', 1]]),
    dict(name = 'Fysik-MC2, cfab', idr = 'O3060133', meters = [['O3060133', 'el_001', 1]]),    
    dict(name = 'Krhuset', idr = 'O3060150_1', meters = [['O3060150', 'el_20412182', 1]]),
    dict(name = 'CTP', idr = 'O3060137', meters = [['O3060137', 'el_20410581', 1],
                                                   ['O3060137', 'el_20323448', 1],
                                                   ['O3060137', 'el_20410579', 1],
                                                   ['O3060137', 'el_20411815', 1],
                                                   ['O3060137', 'el_20411816', 1],
                                                   ['O3060137', 'el_30032339', 1]]),
    dict(name = 'JSP', idr = 'O3060138', meters = [['O3060138', 'el_300029083', 1],
                                                   ['O3060138', 'el_300029084', 1]]),
    dict(name = 'Vasa 1', idr = 'O3060101', meters = [['O3060101', 'el_HMVasa1', 1]]),
    dict(name = 'Vasa 2-3', idr = 'O3060102_3', meters = [['O3060102', 'el_HMVasa2', 1]]),
    dict(name = 'Vasa 4-15', idr = 'O3060104_15', meters = [['O3060100', 'el_20410520', 1],
                                                            ['O3060101', 'el_HMVasa1', -1],
                                                            ['O3060102', 'el_HMVasa2', -1]]),
    dict(name = 'Chabo', idr = 'O4002700', meters = [['O4002700', 'el_2046035', 1]])
         ])

"""UFOn KöldBärare (cooling areas)"""
aKb = list([ dict(name = 'Fysik origo', idr = 'O0007001', meters = [['O0007001', 'kb_901',1]]),
    dict(name = 'Nya Matte', idr = 'O0007006', meters = [['O0007006', 'kb_901', 1]]),
    dict(name = 'Elkraftteknik', idr = 'O0007012', meters = [['O0007012', 'kb_901', 1]]),
    dict(name = 'Chalmers bibliotek', idr = 'O0007017', meters = [['O0007017', 'kb_901', 1]]),
    dict(name = 'Idelra', idr = 'O0007018', meters = [['O0007018', 'kb_901', 1]]),
    dict(name = 'Centrala administrationen', idr = 'O0007019', meters = [['O0007019', 'kb_901', 1]]),
    dict(name = 'Vg och vatten 1', idr = 'O0007023', meters = [['O0007023', 'kb_910', 1]]),
    dict(name = 'EDIT', idr = 'O0007024', meters = [['O0007024', 'kb_901', 1]]),
    dict(name = 'Hrsalar HB', idr = 'O0007025', meters = [['O0007025', 'kb_901', 1]]),
    dict(name = 'Vg och vatten 2', idr = 'O0007027', meters = [['O0007027', 'kb_910', 1]]),
    dict(name = 'Maskinteknik', idr = 'O0007028', meters = [['O0007008', 'kb_501', 0.09]]),
    dict(name = 'Lokalkontor AH', idr = 'O0007888', meters = [['O0007008', 'kb_501', 0.01]]),
    dict(name = 'Fysik Soliden', idr = 'O0011001', meters = [['O0007008', 'kb_501', 0.01]]),
    dict(name = 'Keramforskning', idr = 'O0013001', meters = [['O0007008', 'kb_501', 0.01]]),
    dict(name = 'Kemi- och bioteknik, cfab', idr = 'O3060132', meters = [['O3060132', 'kb_001', 1]]),
    dict(name = 'Fysik MC2', idr = 'O3060133', meters = [[]]),
    dict(name = 'Krhuset', idr = 'O3060150_1', meters = [[]]),
    dict(name = 'CTP', idr = 'O3060137', meters = [['O3060137', 'kb_VMM1', 1],
                                                   ['O3060138', 'kb_VMM1', -1]]),
    dict(name = 'JSP', idr = 'O3060138', meters = [['O3060138', 'kb_VMM1', 1]]),
    dict(name = 'Vasa 1', idr = 'O3060101', meters = [[]]),
    dict(name = 'Vasa 2-3', idr = 'O3060102_3', meters = [[]]),
    dict(name = 'Vasa 4-15', idr = 'O3060104_15', meters = [['O3060104', 'kb_VMM8', 1]]),
    dict(name = 'Chabo', idr = 'O4002700', meters = [[]])
         ])

dc2u.addTSToArea(aFj, b)
dc2u.ensurePTSNonNeg(aFj)
dc2u.addTSToArea(aEl, b)
dc2u.ensurePTSNonNeg(aEl)
dc2u.addTSToArea(aKb, b)
dc2u.ensurePTSNonNeg(aKb)

In [ ]:
"""Cell 5: Write energy signatures of all areas in terms of base, slope for all, day, night"""
importlib.reload(dc2u)
dc2u.writeAreaESigs(aFj, fileName = 'areaESigs20170615.csv')
print('Done')

In [ ]:
"""Cell 6: Write all timeseries from all areas into csv files, overwriting any existing files
If you don't want to write all areas, comment out those you don't want by adding a '#' in front of the line"""
dc2u.collectAllAreaTS(aFj).to_csv('./UFOn/' + 'UFO_fj' + '.csv', sep = ';')
dc2u.collectAllAreaTS(aEl).to_csv('./UFOn/' + 'UFO_el' + '.csv', sep = ';')
dc2u.collectAllAreaTS(aKb).to_csv('./UFOn/' + 'UFO_kb' + '.csv', sep = ';')
print('Wrote area TS to .csv')

In [ ]:
"""Cell 7: Create the summary plot of all parsed areas that have attached timeseriess.
To choose which area list to use, edit the first line (aa = aXX)
To reduce size of this notebook file, image is stored to disk rather than embedded in notebook
Choose name of output file in second line (outputFileName = X.png)"""
aa = aFj
outputFileName = 'areaSummaryIllustration.png'

aPlot = [area for area in aa if 'ts' in area.keys()]

dc2u.ensureOutsideT_area(aPlot, force=False, date_min = None)

meanTCut = None
if 'fj' in aPlot[0]['meters'][0][1]:
    meanTCut = 15
    dc2u.addAreaESigs(aPlot)

nAx = np.ceil( (len(aPlot)))

plt.figure(1);
plt.clf()
plt.setp(plt.gcf(), figHeight=str(2 + 4*nAx), figWidth='16')


for i in range(len(aPlot)):
    p = aPlot[i]
    
    # Temp-sorted plot
    plt.subplot(nAx, 3, (i*3)+1)
    plt.plot(p['ts']['t'], p['ts']['diff'], '.k', alpha = 0.2)
    plt.title(p['name'] + ' - Energy Signature')
    #plt.xlabel('Outside T [C]') 
    plt.ylabel('Energy Load [MW]')
    
    # IF contains ['eSig6'], include esig-lines
    if 'eSig6' in p.keys():
        #pbp = [max(p['ts']['t'])+0.5, meanTCut, min(p['ts']['t']-0.5)]
        plt.plot(p['ppEx'], p['ppESig'][0], 'b', 
                 p['ppEx'], p['ppESig'][1], 'g',
                 p['ppEx'], p['ppESig'][2], 'r')
    
    # Seasonal plot
    plt.subplot(nAx, 3, (i*3)+2)
    # plt.locator_params(axis='x', nticks=4)
    ax = plt.gca()
    hlAvg = dc2u.nPointsAverage(p['ts']['diff'])
    plt.plot(p['ts'].index.values[::24], hlAvg)
    plt.title('id '+p['idr'] + ' - Daily mean load')
    #plt.title('Area ' + p['name'])
    #plt.xlabel('Outside T [C]') 
    ax.set_xticks(ax.get_xticks()[[0,4,-1]])
    
    # Hourly variance plot
    plt.subplot(nAx, 3, (i*3)+3)
    ax = plt.gca()
    dc2u.addHourMean(p, tCutT = meanTCut, overWrite = True)
    plt.plot(p['hourMean'])
    plt.title('Daily profile by hour')
    #plt.xlabel('Hour of day') 
    
    ax.grid(b=True, which='both', ls=':')
    plt.setp(ax, xticks = np.linspace(1,23,12))
    
plt.savefig(outputFileName)
print('Done')

In [ ]:
"""Cell 8: Display summary information about parsed files
b.pCond(criteriaList) is a generator which yields all meter streams for which 
'accessPath' contains each element in 'criteriaList'. 
For example, b.pCond(['O000','fj','Energy']) will yield all district heating meters 
from Academiska Hus (they all begin with O000)
"""
for p in b.pCond(['O', 'fj']): 
    print('building '+p['buildingName']+', meter '+str(p['accessPath'][:2])+'. Date span = '+str(p['ts'].index[0])+' - '+str(p['ts'].index[-1])+'.')


In [12]:
"""Cell 9: Summary information of areas
Select medium (fj, el, kb) by editing the first line"""
atmp = aFj

for ar in atmp:
    if not 'ts' in ar.keys():
        print('Area '+ar['idr']+', '+ar['name']+'. No timeseries data for medium.')
    else:
        print('Area '+ar['idr']+', '+ar['name']+'. Date span = '+str(ar['ts'].index[0])+' - '+str(ar['ts'].index[-1])+'.')

area O0007001, Fysik origo. Date span = 2015-01-01 01:00:00 - 2017-03-23 06:00:00.
area O0007005, Polymerteknologi. Date span = 2015-06-04 16:00:00 - 2017-03-23 06:00:00.
area O0007006, Nya Matte. Date span = 2015-02-24 14:00:00 - 2017-03-23 07:00:00.
area O0007012, Elkraftteknik. Date span = 2015-06-04 17:00:00 - 2016-07-01 00:00:00.
area O0007014, Gibraltar herrgrd. No timeseries data for medium.
area O0007017, Chalmers bibliotek. Date span = 2016-05-23 10:00:00 - 2017-03-23 08:00:00.
area O0007018, Idelra. Date span = 2015-09-30 02:00:00 - 2017-03-23 09:00:00.
area O0007019, Centrala administrationen. Date span = 2015-06-05 09:00:00 - 2017-03-23 12:00:00.
area O0007021, Hrsalar HC. Date span = 2016-05-24 14:00:00 - 2017-03-29 08:00:00.
area O0007022, Hrsalar HA. Date span = 2015-04-27 16:00:00 - 2017-03-29 09:00:00.
area O0007023, Vg och vatten 1. Date span = 2013-01-01 00:00:00 - 2017-03-29 08:00:00.
area O0007024, EDIT. Date span = 2015-09-18 09:00:00 - 2017-03-29 10:00:00.
area O